## Introduction

## Who am I?

## Where do I work?

## What do I do?

## Bulk sequencing data

- 1953 - DNA structure
- 1953 - First protein amino acid sequencing
- 1975 - Sanger sequencing
- 1985 - First PCR
- 1988 - PCR with Taq polymerase
- 1988 - Automation of Sanger sequencing
- 1993 - First qPCRs
- 1996 - Pyrosequencing
- 1995 - Microarrays
    - Quantify gene expression using a chip
- 2001 - First draft of the human genome
- 2003 - First NGS DNA sequencer
- 2005 - Next-generation sequencing
- 2009 - First single-cell RNA-seq [@tang2009mrna]
- 2010 - RNA-seq
- 2013 - Single-cell RNA-seq is declared method of the year

[@zhu2020pcr; @gondane2023revealing]

## Bulk RNA-seq

- RNA-seq is a method to quantify gene expression in a sample by sequencing the RNA molecules in the sample.
- Can be used to quantify gene expression, identify alternative splicing, identify novel genes, quantify allele-specific expression, etc.
- Can estimate the **average expression** level for each gene across a population of cells.

## Single-cell RNA-seq

- scRNA-seq methods can quantify gene expression in individual cells.
- Can be used to study cell  heterogeneity, sunch as identify cell types, cell states, heterogeneity, and cell-cell interactions.
- Suffers from drop-out, higher technical noise and lower genomic resolution compared to bulk RNA-seq.

## Sample representation

- In bulk data, each sample is repressented by a vector, where each value is a gene.
- In single cell data, each sample is a matrix, where each row is a gene and each column is a cell.

\begin{align}
Bulk &= \begin{bmatrix} gene_{1} \\ gene_{2} \\ gene_{3}\\ \vdots \\ gene_{n} \end{bmatrix} 
\\
\\

Single-cell &= \begin{bmatrix}
    gene_1, cell_1 & gene_1, cell_2 & gene_1, cell_3 & \dots  & gene_1, cell_m \\
    gene_2, cell_1 & gene_2, cell_2 & gene_2, cell_3 & \dots  & gene_2, cell_m \\
    gene_3, cell_1 & gene_3, cell_2 & gene_3, cell_3 & \dots  & gene_3, cell_m \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    gene_n, cell_1 & gene_n, cell_2 & gene_n, cell_3 & \dots  & gene_n, cell_m
\end{bmatrix}

\end{align}


## Single-cell vs bulk RNAseq

|Feature              | Bulk data                          | Single-cell data                  |
|---------------------|------------------------------------|-----------------------------------|
| Cell resolution     | Average of all cells               | Individual cell resolution        |
| Sample representation | Vector of gene expression values | Matrix of gene expression values  |
| Genomic resolution  | Higher, depends on sequencing depth  | Lower, depends on starting material |
| Cost                | Lower                                | High                              |
| Computational requirements | Lower                        | Higher                              |
| Data size           | Lower                               | Higher                           |
| Data interpretation | Simple                             | Complex                           |

## Scanpy (Python) vs Seurat (R)

- Both are: 
    - User-friendly tools for single-cell analysis
    - Open source 
    - Well-documented (Maybe Seurat more than Scanpy)
    - Widely-used
- Choice depends on:
    - Language preference
    - Team expertise
    - Integration with downstream analysis
    - Speed and memory requirements (In general, Python is faster and more memory efficient than R)

> Hint: A good bioinformatician is not restricted by language. You can use R in Python can be done using the `rpy2` package. And Python can be use within R using `reticulate`. 

## Scale of scRNAseq data

![Number of cells per study over years    [@svensson2020curated]](number_of_cells.png)

## AnnData object

![AnnData object, source: scanpy web](https://falexwolf.de/img/scanpy/anndata.svg) 

## Set up

- Install Miniconda
- Create a new environment

```bash
conda create -n myscanpy python=3.10
conda activate myscanpy
pip install -r requirements.txt
quarto check jupyter

```

## Import libraries


In [ ]:
import scanpy as sc
import scipy as sp
import numpy as np
import os
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import fast_matrix_market

Scanpy setttings

In [ ]:
sc.settings.verbosity = 3   # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

## Download data

```bash
wget https://ndownloader.figshare.com/files/10038325
unzip 10038325
wget https://ndownloader.figshare.com/files/10038328
mv 10038328 droplet_metadata.csv
wget https://ndownloader.figshare.com/files/10039264
mv 10039264 droplet_annotation.csv
```


## Load data

Load 1 dataset


In [ ]:
path = "droplet/Bladder-10X_P4_3/"
adata = sc.read_10x_mtx(path, var_names='gene_symbols', cache=True)
adata

> Hint: When single-cell datasets are large, you can use `polars` instead of `pandas` to read tabular data. You can also use `fast_matrix_market` to read mtx files.


In [ ]:
#| code-fold: true

def read_scrnaseq_faster(
   path: str
   )-> sc.AnnData:
    """
    Read a sparse matrix in Matrix Market format and two CSV files with gene and cell metadata
    into an AnnData object.
    
    Args:
        path: Path to the directory containing the matrix.mtx, genes.tsv, and barcodes.tsv files.
        
    Returns:
        An AnnData object with the matrix, gene metadata, and cell metadata.

    """
    mtx_file = os.path.join(path, "matrix.mtx")
    gene_info = os.path.join(path, "genes.tsv")
    cell_metadata = os.path.join(path, "barcodes.tsv")
    
    # Read the .mtx file into a sparse matrix using the fast_matrix_market package (master than scanpy, uses multiprocessing)
    mtx = fast_matrix_market.mmread(mtx_file)

    # Convert the sparse matrix to a CSR matrix
    # Otherwise you will not be able to use it with scanpy
    if isinstance(mtx, sp.sparse.coo.coo_matrix):
        mtx = mtx.tocsr()
    
    # Create an AnnData object
    adata = sc.AnnData(X=mtx.T)

    # Polars is faster than pandas for reading test files
    # Read the gene names and cell names into the AnnData object
    
    adata.var = pl.read_csv(gene_info, separator= '\t', has_header=False).to_pandas()
    
    # Read the cell names and cell metadata into the AnnData object
    adata.obs = pl.read_csv(cell_metadata, separator= '\t', has_header=False).to_pandas()
    return adata

Load 7 datasets


In [ ]:
# Get folders in droplet
folders = os.listdir("droplet")
adata = []
for folder in folders[0:10]:
    channel = folder.split("-")[1]
    tissue = folder.split("-")[0]
    path = f"droplet/{folder}/"
    adata_small = sc.read_10x_mtx(path, var_names='gene_symbols', cache=True)
    adata_small.obs["channel"] = channel
    adata_small.obs["tissue"] = tissue
    if folder == folders[0]:
        adata = adata_small
    else:
        adata = adata.concatenate(adata_small)
    print(f"Loaded {folder}")
adata

## Doublets

## Preprocessing

### Highest expressing genes
Look for suspects: MALAT1, mitochondrial genes, ribosomal genes, componenets of the cytoskeleton, etc.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

### Quality metrics

`sc.pp.calculate_qc_metrics` computes quality control metrics for each cell.
- Number of counts per cell
- Number of genes per cell
- Percentage of counts that come from mitochondrial genes.


In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('Mt')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

Different sampes might require different thresholds


In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True, groupby='tissue', rotation = 90)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt', color='tissue')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='tissue')

## Filtering

Filter cells based on quality metrics


In [ ]:
sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_genes(adata, min_cells=5)
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

## Normalisation: size normalisation and log transformation

For simplicity, we will use the simplest method: library size normalisation and log transformation, but there are others
[@risso2018general, @hafemeister2019normalization]. 

Steps:

- Transcript length normalisation: 
    - Adjusts for differences in transcript length between genes
    - Divides the counts in each cell by the length of the transcript
    - Not necessary if you sequence a fixed region of the transcript ( 3' or 5' in 10X, our case)
- Library size normalisation: 
    - Adjusts for differences in sequencing depth between cells
    - Divides the counts in each cell by the total counts in that cell and multiplies by a scale factor (e.g. 10,000)

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

- Log transformation:
    - Logarithm of the normalised counts
    - Makes the data more normally distributed

In [ ]:
sc.pp.log1p(adata)

## Highly variable genes


In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

Set the .raw attribute of the AnnData object to the normalized and logarithmized raw gene expression for later use in differential testing and visualizations of gene expression. This simply freezes the state of the AnnData object.

You can get back an AnnData of the object in .raw by calling .raw.to_adata().

In [ ]:
adata.raw = adata

If you don’t proceed below with correcting the data with sc.pp.regress_out and scaling it via sc.pp.scale, you can also get away without using .raw at all.

The result of the previous highly-variable-genes detection is stored as an annotation in .var.highly_variable and auto-detected by PCA and hence, sc.pp.neighbors and subsequent manifold/graph tools. In that case, the step actually do the filtering below is unnecessary, too.

Actually do the filtering


In [ ]:
adata = adata[:, adata.var.highly_variable]
adata

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.


In [ ]:
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

Scale each gene to unit variance. Clip values exceeding standard deviation 10.

In [ ]:
sc.pp.scale(adata, max_value=10)

## Principal component analysis
Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.


In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color='tissue')

Varinace explained by each component

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

## Neighborhood graph

Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. You might simply use default values here. For the sake of reproducing Seurat’s results, let’s take the following values.


In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

We suggest embedding the graph in two dimensions using UMAP (McInnes et al., 2018), see below. It is potentially more faithful to the global connectivity of the manifold than tSNE, i.e., it better preserves trajectories. In some ocassions, you might still observe disconnected clusters and similar connectivity violations. They can usually be remedied by running:


In [ ]:
 # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata, init_pos='paga')
sc.tl.umap(adata)

As we set the .raw attribute of adata, the previous plots showed the “raw” (normalized, logarithmized, but uncorrected) gene expression. You can also plot the scaled and corrected gene expression by explicitly stating that you don’t want to use .raw.

In [ ]:
sc.pl.umap(adata, color='tissue', use_raw=False)

## Clustering the neighborhood graph
As with Seurat and many other frameworks, we recommend the Leiden graph-clustering method (community detection based on optimizing modularity) by Traag et al. (2018). Note that Leiden clustering directly clusters the neighborhood graph of cells, which we already computed in the previous section.


In [ ]:
sc.tl.leiden(adata, resolution=0.02)
sc.pl.umap(adata, color='leiden', use_raw=False)

## Finding marker genes

Let us compute a ranking for the highly differential genes in each cluster. For this, by default, the .raw attribute of AnnData is used in case it has been initialized before. The simplest and fastest method to do so is the t-test.


In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)

During the course of this analysis, the AnnData accumlated the following annotations.


In [ ]:
adata

## What is the goal of this workshop?

## What is single-cell data?

What is single-cell RNA-seq and how does it compare to bulk RNA-seq?
What are some of the typical applications of scRNA-seq?
How are samples typically prepared for scRNA-seq?
What are the differences between some of the most popular protocols and what are their advantages and disadvantages?
What experimental design choices should be considered in scRNA-seq?
What are some of the challenges of scRNA-seq data compared to bulk data?




## Inspirations

- [Analysis of single cell RNA-seq data](https://www.singlecellcourse.org/)
    - Course from University of Cambridge Bioinformatics training unit
- [Single cell study database](www.nxn.se/single-cell-studies/)
- [Scanpy tutorials](https://scanpy-tutorials.readthedocs.io/en/latest/)